### Imports

In [30]:
import numpy as np
import pandas as pd

import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as MAE
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import absolute

### Dataset preprocessing 

In [31]:
df = pd.read_csv("data/covid_data_train.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)

#### Removing duplicates & rows with missed target variable

In [32]:
df = df.drop_duplicates(subset='name', keep="last")
df = df[df.inf_rate.isnull() == False]

In [33]:
df.head()

,lat,lng,name,population,district,subject,density,ivl_per_100k,ivl_number,ekmo_per_100k,...,epirank_avia,epirank_bus,epirank_train,epirank_avia_cat,epirank_bus_cat,epirank_train_cat,whole_population,urban,rural,has_metro
0,52.651055,90.101159,Абаза,17111.0,Сибирский,Хакасия,8.68,NaN,NaN,NaN,...,NaN,2.278095e-11,NaN,NaN,0.0,NaN,64241.0,44921.0,19320.0,0.0
1,53.720902,91.442435,Абакан,165183.0,Сибирский,Хакасия,8.68,NaN,NaN,NaN,...,0.000903,8.343086e-10,0.001383,0.0,1.0,0.0,64241.0,44921.0,19320.0,0.0
2,53.682800,53.655701,Абдулино,20663.0,Приволжский,Оренбургская область,15.95,27.7,542.0,0.05,...,NaN,5.164813e-14,0.000376,NaN,0.0,0.0,270081.0,156761.0,113320.0,0.0
3,44.864953,38.157819,Абинск,34926.0,Южный,Краснодарский край,73.73,17.6,1000.0,0.07,...,NaN,1.187676e-14,0.000186,NaN,0.0,0.0,847286.0,452437.0,394849.0,0.0
4,56.525460,52.997251,Агрыз,19299.0,Приволжский,Татарстан,57.27,28.2,1100.0,0.00,...,NaN,4.750316e-13,0.001003,NaN,0.0,0.0,527462.0,371965.0,155497.0,0.0


#### Deciding what columns to take
Согласно https://docs.google.com/document/d/1BdrdrtFyiVLmMER_H4e2BxsyiMZzP3Pipsq2bZHnXjo/edit?usp=sharing (Feature Importance после использования Random Forest Regressor) было решено не использовать категориальные переменные - ___name, district, subject, region_x___, во-первых, потому что их значимость оказалось очень низкой, а во-вторых, потому что по идее результат предсказания не должен зависеть от названия населенного пункта, что и подтвердилось ранее. Также было решено отказаться от бинарной переменной ***___has_metro___***, потому что она имеет маленькую значимость.  
Поэтому берем только numerical колонки для обучения.

In [34]:
numerical_df = df.select_dtypes(include=np.number).drop('has_metro', axis=1)

#### Deleting columns containing >= 50% NaN values (optional)

In [35]:
# Delete columns containing either 50% or more than 50% NaN Values
perc = 50.0
min_count =  int(((100-perc)/100)*numerical_df.shape[0] + 1)
mod_df = numerical_df.dropna( axis=1, 
                thresh=min_count)

numerical_df = mod_df
numerical_df.head()

,lat,lng,population,density,ivl_per_100k,ivl_number,ekmo_per_100k,inf_rate,avg_temp_min,avg_temp_max,...,num_phones_urban_2019,bus_march_travel_18,bus_april_travel_18,epirank_bus,epirank_train,epirank_bus_cat,epirank_train_cat,whole_population,urban,rural
0,52.651055,90.101159,17111.0,8.68,NaN,NaN,NaN,1.386294,-12.29,5.57,...,16199.99,21414.0,22186.0,2.278095e-11,NaN,0.0,NaN,64241.0,44921.0,19320.0
1,53.720902,91.442435,165183.0,8.68,NaN,NaN,NaN,1.386294,-12.29,5.57,...,16199.99,21414.0,22186.0,8.343086e-10,0.001383,1.0,0.0,64241.0,44921.0,19320.0
2,53.682800,53.655701,20663.0,15.95,27.7,542.0,0.05,2.079442,-6.71,5.71,...,106326.00,135226.1,133554.9,5.164813e-14,0.000376,0.0,0.0,270081.0,156761.0,113320.0
3,44.864953,38.157819,34926.0,73.73,17.6,1000.0,0.07,2.890372,5.14,16.86,...,315344.00,329792.1,328273.8,1.187676e-14,0.000186,0.0,0.0,847286.0,452437.0,394849.0
4,56.525460,52.997251,19299.0,57.27,28.2,1100.0,0.00,2.708050,-2.14,5.00,...,144303.99,157885.0,162064.9,4.750316e-13,0.001003,0.0,0.0,527462.0,371965.0,155497.0


In [36]:
# Optional filling NaN in modified df
for i in mod_df.columns[mod_df.isnull().any(axis=0)]: 
    mod_df[i].fillna(mod_df[i].mean(),inplace=True)
numerical_df = mod_df

#### Train/Test split
Сразу делаем разбиение на тестовую и тренировочную выборки. 
Это связано с выбором дальнейшей модели для обучения. Было решено далее работать с бустинг-моделями, в нашем случае - с xgboost, так как согласно данной статье https://link.medium.com/X5egHCVBHrb, они могут и олично работают с данными в которых есть пропущенные значения, и стратегия использовать именно бустинг-модели позволяет работать с неполным набором признаков (то есть, если в реальной ситуации окажется что некоторые признаки, необходимые для получения предсказания отсутствуют, то их значения можно просто заменять NaN и подавать на вход модели, поэтому использование бустинг-моделей и становится рациональным, потому что реальная ситуация не идеальна и многие признаки могут просто отсутствовать). Кроме того, использование бустинг-моделей не предполагает стандартизации данных https://stats.stackexchange.com/questions/485677/does-xgboost-require-standarized-data, что также является плюсом.

In [37]:
y = numerical_df.inf_rate
numerical_df.drop("inf_rate", axis=1, inplace=True)

In [38]:
numerical_df.shape, y.shape

((395, 100), (395,))

In [39]:
X = numerical_df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=2019)

### Training model (xgBoost)

#### Getting cross_val_score on train data 

In [40]:
model1 = XGBRegressor(eval_metric='mae')
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model1, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean MAE: 0.021 (0.022)


#### Getting MAE score on test data

In [41]:
model = XGBRegressor(eval_metric='mae')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(MAE(y_test, y_pred))

0.006910315506914175


#### Getting cross_val_score on all data

In [42]:
model2 = XGBRegressor(eval_metric='mae')
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model2, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean MAE: 0.014 (0.017)


#### Fitting on all data

In [44]:
model = XGBRegressor(eval_metric='mae')
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='mae', gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

#### Saving model

In [45]:
joblib.dump(model, "XGBmodel.joblib") 

['XGBmodel.joblib']